# North Macedonian 2024 Election Fact Checking Pipeline
---
This notebook is part of the North Macedonian 2024 Election Misinformation Assessment project. It contains code to analyze and visualize misinformation related to the elections in North Macedonia.

##### Libraries

In [1]:
from dotenv import load_dotenv
import requests
import json
from urllib.parse import quote_plus
import tqdm
import json
import os
import csv
import pandas as pd

##### Constants

In [2]:
load_dotenv()
llama3_url = "http://localhost:11434/api/generate"
google_api = os.getenv("GOOGLE-API-KEY")
cx = os.getenv("CX")

dataset = "Dataset/VMRO-DPMNE_ad_creative_bodies.csv"
party = "VMRO-DPMNE"

## Data Collection

In [7]:
pandas_df = pd.read_csv(dataset)
print(pandas_df.head())

      ad_archive_id                                 ad_creative_bodies
0   427480589881211  Para disa ditëve në emisionin “Politiko” e kam...
1  1636678057070325  Промените се во твои раце!\nМакедонија повторн...
2   252100357930225                        За многу години Цветници!!!
3  1117244869521215  ✒️Shkruan: @[100005900348509:2048:Enis Shaqiri...
4  1595858924569626  ИЗГУБЕНА Е ДОВЕРБАТА ВО ПОЛИТИЧАРИТЕ БИДЕЈЌИ Н...


In [3]:
parties = ["DUI", "SDSM", "Left", "VMRO-DPMNE"]
parties_csv_name = ["DUI.csv", "meta-ad-library-2025-04-30_2-SDSM.csv", "meta-ad-library-2025-04-30_3-Левица.csv", "meta-ad-library-2025-04-30-VMRO-DPMNE.csv"]
parties_csv_name_dict = {
    "DUI": "DUI.csv",
    "SDSM": "meta-ad-library-2025-04-30_2-SDSM.csv",
    "Left": "meta-ad-library-2025-04-30_3-Левица.csv",
    "VMRO-DPMNE": "meta-ad-library-2025-04-30-VMRO-DPMNE.csv"
}

## Data Preprocessing

### Extract Claims

In [24]:

def extract_claims(text, id, iteration):
    promptQuestion= """For the bellow text, extract at max 3 claims and give to me in the JSON format. The claims should in English Language. You should only return the json. Do not add any other text. The JSON should only have 1 key "Claims" and the value should be a list of claims."""
    
    extract_claim_payload = {
    "model": "llama3",
    "prompt": promptQuestion + text,
    }
    
    extract_claim_response = requests.post(llama3_url, json=extract_claim_payload)
    
    if extract_claim_response.status_code == 200:
        lines = extract_claim_response.text.splitlines()
        output = ""
        for line in lines:
            try:
                # Parse each line as JSON
                line_data = json.loads(line)
                # Concatenate the "response" field
                output += line_data.get("response", "")
            except json.JSONDecodeError:
                # Skip lines that are not valid JSON
                continue
        # Parse the final concatenated output as JSON
        try:
            final_output = json.loads(output)
            
            print("Extracted Claims: Iteration:", iteration, "ID:", id, "Claims:", final_output)
            return final_output
        except json.JSONDecodeError:
            # Add a } to the end of the output
            output += "}"
            try:
                final_output = json.loads(output)
                print("Extracted Claims: Iteration:", iteration, "ID:", id, "Claims:", final_output)
                return final_output
            except json.JSONDecodeError:
                print("\n")
                print("Final output is not valid JSON: Iteration:", iteration, "ID:", id, "Output:", output)
                print("\n")
                return None
    else:
        print(f"Error: {extract_claim_response.status_code}")
        print(extract_claim_response.text)

In [19]:
ids_to_claims = {}
read_csv_path_bodies = "./dataset/ad-creative-bodies-extraction/"

for party in parties:
    # Read the CSV file into a DataFrame
    csv_file_path = os.path.join(read_csv_path_bodies, party)
    csv_file_path = csv_file_path + "_ad_creative_bodies.csv"
    # Add "ad_creative_bodies" in front of the file name
    # csv_file_path = csv_file_path.replace(".csv", "_ad_creative_bodies.csv")
    print(csv_file_path)
    pandas_df = pd.read_csv(csv_file_path)
    
    # Print the DataFrame shape
    print(f"Shape of {party} DataFrame:", pandas_df.shape)
    
    # Check if the DataFrame is empty
    if pandas_df.empty:
        print(f"The DataFrame for {party} is empty.")
        continue
    
    # Iterate through the rows of the DataFrame


# for index, row in tqdm.tqdm(pandas_df.iterrows(), total=pandas_df.shape[0]):
#     # Extract the text from the DataFrame
#     text = row['ad_creative_bodies']
#     id = row['ad_archive_id']
#     # Extract claims from the text
#     claims = extract_claims(text)
#     # Store the claims in the dictionary
#     ids_to_claims[id] = claims
    
# # Check if the folder and file exist
# if not os.path.exists("./PreprocessedDataset/"):
#     os.makedirs("./PreprocessedDataset/")
    

# # Save the claims to a CSV file
# with open("./PreprocessedDataset/" + party + "_claims.csv", "w", newline='', encoding='utf-8') as csvfile:
#     fieldnames = ['ad_archive_id', 'claims']
#     writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
#     writer.writeheader()
#     for id, claims in ids_to_claims.items():
#         writer.writerow({'ad_archive_id': id, 'claims': json.dumps(claims)})

./dataset/ad-creative-bodies-extraction/DUI_ad_creative_bodies.csv
Shape of DUI DataFrame: (102, 2)
./dataset/ad-creative-bodies-extraction/SDSM_ad_creative_bodies.csv
Shape of SDSM DataFrame: (60, 2)
./dataset/ad-creative-bodies-extraction/Left_ad_creative_bodies.csv
Shape of Left DataFrame: (86, 2)
./dataset/ad-creative-bodies-extraction/VMRO-DPMNE_ad_creative_bodies.csv
Shape of VMRO-DPMNE DataFrame: (28, 2)
./dataset/ad-creative-bodies-extraction/ZNAM_ad_creative_bodies.csv
Shape of ZNAM DataFrame: (1, 2)


In [26]:
save_folder_path = "./dataset/claims-extraction/"

# Extract the Claims through LLM
for party in tqdm.tqdm(parties):
    # Read the CSV file into a DataFrame
    csv_file_path = os.path.join(read_csv_path_bodies, party)
    csv_file_path = csv_file_path + "_ad_creative_bodies.csv"
    # Add "ad_creative_bodies" in front of the file name
    # csv_file_path = csv_file_path.replace(".csv", "_ad_creative_bodies.csv")
    print(csv_file_path)
    pandas_df = pd.read_csv(csv_file_path)
    
    # Print the DataFrame shape
    print(f"Shape of {party} DataFrame:", pandas_df.shape)
    
    # Check if the DataFrame is empty
    if pandas_df.empty:
        print(f"The DataFrame for {party} is empty.")
        continue
    
    iteration = 1
    # Iterate through the rows of the DataFrame
    for index, row in pandas_df.iterrows():
        # Extract the text from the DataFrame
        text = row['ad_creative_bodies']
        text = str(text)
        
        id = row['ad_archive_id']
        id = str(id)
        
        # Extract claims from the text
        claims = extract_claims(text, id, iteration)
        # Store the claims in the dictionary
        ids_to_claims[id] = claims
        iteration += 1
    # Save the claims to a CSV file
    with open(os.path.join(save_folder_path, party + "_claims.csv"), "w", newline='', encoding='utf-8') as csvfile:
        fieldnames = ['ad_archive_id', 'claims']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for id, claims in ids_to_claims.items():
            writer.writerow({'ad_archive_id': id, 'claims': json.dumps(claims)})
    ids_to_claims = {}
    

  0%|          | 0/5 [00:00<?, ?it/s]

./dataset/ad-creative-bodies-extraction/DUI_ad_creative_bodies.csv
Shape of DUI DataFrame: (102, 2)
Extracted Claims: Iteration: 1 ID: 6795509300551996 Claims: {'Claims': ['DUI will go into opposition', 'DUI will respond to crime and corruption']}
Extracted Claims: Iteration: 2 ID: 746708557578780 Claims: {'Claims': ['Three platforms are available for the upcoming parliamentary elections: Tirana platform, Sofia platform, and Macedonian platform.', 'The left will coalition with a government that accepts red lines in defending the Macedonian state.', 'Macedonians will put an end to the dictatorships and defeats of the SDS and DUI regime and restore the Republic.']}
Extracted Claims: Iteration: 3 ID: 464195942757638 Claims: {'Claims': ['The movement ZNAK for Our Macedonia is clear in its stance that it is against the opening of mines in Strumica.', "It should not be allowed to trade with people's health.", 'There needs to be an investigation whether DUI functionaries traded influence.']}


 20%|██        | 1/5 [02:33<10:13, 153.27s/it]

Extracted Claims: Iteration: 102 ID: 1585098798694924 Claims: {'Claims': ['Demokrati claims that the calculations of narco-bands indicate a brutal election.', 'Different political parties and their governing bodies over the past three decades have created and nurtured dangerous criminal bands which now take revenge on their own.', 'The spiral of violence must be stopped because it is consuming us.']}
./dataset/ad-creative-bodies-extraction/SDSM_ad_creative_bodies.csv
Shape of SDSM DataFrame: (60, 2)
Extracted Claims: Iteration: 1 ID: 988735132879405 Claims: {'Claims': ['There is no giving up on destiny', 'SDSM', 'Aerodrome', 'Macedonia']}
Extracted Claims: Iteration: 2 ID: 1138731707263665 Claims: {'Claims': ['SVM is the only party prepared to fight for citizens and the future of the state.', 'We are united and strong, we have endured time and weather, good and bad, and always been on the guard of the homeland.', "It's not an option for us to give up. We're a party made of solid materi

 40%|████      | 2/5 [03:57<05:38, 112.81s/it]

...

Extracted Claims: Iteration: 86 ID: 3593778017563269 Claims: ['NGOs are pijavici (scammers) who circulate public funds to spread Western propaganda', 'The City of Skopje, specifically the Sector for International Cooperation, should have a special program for cooperation with NGOs and foundations', 'It is not enough that NGOs are involved in the implementation of every program and activity of the City of Skopje']
./dataset/ad-creative-bodies-extraction/VMRO-DPMNE_ad_creative_bodies.csv
Shape of VMRO-DPMNE DataFrame: (28, 2)
Extracted Claims: Iteration: 1 ID: 427480589881211 Claims: {'Claims': ['The anti-Albanian quarters have a tendency to abolish amnesty and obscure the fight for freedom and justice of UÇK.', "Gordana Siljanovska's words confirmed that VMRO-DPMNE's political battle is to subjugate Albanians, not continue the partnership built since 2001", "EU Front should gain more than 22 mandates and 'Siljanovskas' should remain only a memory"]}
Extracted Claims: Iteration: 2 ID: 16

 80%|████████  | 4/5 [06:31<01:23, 83.55s/it] 

Extracted Claims: Iteration: 28 ID: 369290982662369 Claims: [{'Claims': ['President Stevo Pendarovski will run for a second term with support from LSDM.', 'Gordana Siljanovska Davkova, a deputy from VMRO-DPMNE, will also run again after five years.', 'Arben Taravari has been officially promoted as a presidential candidate by the VLEN coalition.']}]
./dataset/ad-creative-bodies-extraction/ZNAM_ad_creative_bodies.csv
Shape of ZNAM DataFrame: (1, 2)


100%|██████████| 5/5 [06:32<00:00, 78.47s/it]



Final output is not valid JSON: Iteration: 1 ID: 7582091031857061 Output: [
"Claims": [
"What attributes should the next President of North Macedonia possess?",
"The candidates have started to offer a wide range of promises and slogans, all with the aim of attracting voters.",
"There is no concrete answer as to what these candidates are offering."
]
]}




In [104]:
# Remove the columns which have none in the claims
for party in parties:
    # Read the CSV file into a DataFrame
    csv_file_path = os.path.join(save_folder_path, party + "_claims.csv")
    pandas_df = pd.read_csv(csv_file_path)
    print("Before removing the columns for the Party " + party + " the shape is: ", pandas_df.shape)
    
    
    # Check if the DataFrame is empty
    if pandas_df.empty:
        print(f"The DataFrame for {party} is empty.")
        continue
    
    # Remove the rows where claims are None
    pandas_df = pandas_df[pandas_df['claims'].notna()]

    print("After removing the columns for the Party " + party + " the shape is: ", pandas_df.shape)
    print("\n")
    
    # Save the cleaned DataFrame to a new CSV file
    cleaned_csv_file_path = os.path.join(save_folder_path, party + "_claims_cleaned.csv")
    pandas_df.to_csv(cleaned_csv_file_path, index=False)

Before removing the columns for the Party DUI the shape is:  (148, 2)
After removing the columns for the Party DUI the shape is:  (135, 2)


Before removing the columns for the Party SDSM the shape is:  (60, 2)
After removing the columns for the Party SDSM the shape is:  (58, 2)


Before removing the columns for the Party Left the shape is:  (86, 2)
After removing the columns for the Party Left the shape is:  (82, 2)


Before removing the columns for the Party VMRO-DPMNE the shape is:  (28, 2)
After removing the columns for the Party VMRO-DPMNE the shape is:  (26, 2)


Before removing the columns for the Party ZNAM the shape is:  (1, 2)
After removing the columns for the Party ZNAM the shape is:  (0, 2)




### Claim to Questions

In [36]:
def claim_to_question(claim):
    prompt_question = f""" For the bellow claim generate me 3 neural fact-checking questions and give to me in the format of a json. You should only return the json. The json should have only 1 key `Questions` and the value should be a list of questions.
    Claim: "{claim}"""
    
    extract_claim_payload = {
    "model": "llama3",
    "prompt": prompt_question + text,
    }
    
    question_response = requests.post(llama3_url, json=extract_claim_payload, timeout=30)
    
    if question_response.status_code == 200:
        # Process the streamed response
        lines = question_response.text.splitlines()
        output = ""

        for line in lines:
            try:
                # Parse each line as JSON
                line_data = json.loads(line)
                # Concatenate the "response" field
                output += line_data.get("response", "")
            except json.JSONDecodeError:
                # Skip lines that are not valid JSON
                continue

        # Parse the final concatenated output as JSON
        try:
            final_output = json.loads(output)
            return final_output
        except json.JSONDecodeError:
            # Add a } to the end of the output
            output += "}"
            try:
                final_output = json.loads(output)
                return final_output
            except json.JSONDecodeError:
                return None
    else:
        print(f"Error: {question_response.status_code}")
        print(question_response.text)

In [105]:
# Open the Claims and read everything once to make sure that the json is properly formated
for party in parties:
    # Read the CSV file into a DataFrame
    csv_file_path = os.path.join(save_folder_path, party + "_claims_cleaned.csv")
    pandas_df = pd.read_csv(csv_file_path)
    print(pandas_df.head())
    
    
    # # Iterate through the rows of the DataFrame
    # for index, row in pandas_df.iterrows():
    #     # Extract the text from the DataFrame
    #     claims = row['claims']
    #     claims = json.loads(claims)
    #     print(claims)
    
    # Iterate through the claims. If the claim is found which is an array, add it to the dictionary
    for index, row in pandas_df.iterrows():
        # Extract the text from the DataFrame

        try:
            claims = row['claims']
            claims = json.loads(claims)
            c = claims["Claims"]
        except TypeError:
            
            if len(claims) == 1:
                print("Array Error")
                # Update the dataframe
                pandas_df.at[index, 'claims'] = claims[0]
                print(pandas_df.at[index, 'claims'])
            
            else:
                
                # Add the claim to the dictionary
                print("No Claims field error")
                # Update the dataframe
                claims = {"Claims": claims}
                pandas_df.at[index, 'claims'] = claims

        
        
    
    # Save the cleaned DataFrame to a new CSV file
    cleaned_csv_file_path = os.path.join(save_folder_path, party + "_claims_cleaned.csv")
    pandas_df.to_csv(cleaned_csv_file_path, index=False)
    


      ad_archive_id                                             claims
0   988735132879405  {"Claims": ["\u041d\u0435\u043c\u0430\u041e\u0...
1  1138731707263665  {"Claims": ["SDSM is the only party ready to f...
2  1462600661315120  {"Claims": ["SOSM is the only party ready to f...
3  1168468637617703  {"Claims": ["It is not too late to change dire...
4  1366791877252741  {"Claims": ["We are striving for great support...
No Claims field error
Array Error
{'Claims': ['The public-private partnership agreement for the Smart Airport project worth 10.7 million euros was signed on March 25, 2024.', 'There is no mention of digitization of municipal administrative services in the agreement.', 'The agreement does not specify how it will stimulate innovation and startup businesses.']}
No Claims field error
No Claims field error
Array Error
{'Claims': ['The government led by SDSM has separated 700,000 denari for the citizens of Aerodrom municipality to work on technical documentation for buildin

In [111]:
# Read the Claims and each claim and if there is an error remove the row

for party in parties:
    # Read the CSV file into a DataFrame
    csv_file_path = os.path.join(save_folder_path, party + "_claims_cleaned.csv")
    pandas_df = pd.read_csv(csv_file_path)
    print("Shape Before Filtering: ", pandas_df.shape)
    
    # Iterate through the rows of the DataFrame
    for index, row in pandas_df.iterrows():
        # Extract the text from the DataFrame
        claims = row['claims']
        try:
            claims = json.loads(claims)
            c = claims["Claims"]
        except:
            # Remove the row from the DataFrame
            pandas_df = pandas_df.drop(index)
            print("Removed the row with index: ", index)
            continue
    
    print("Shape After Filtering: ", pandas_df.shape)
    
    # Save the cleaned DataFrame to a new CSV file
    cleaned_csv_file_path = os.path.join(save_folder_path, party + "_claims_cleaned.csv")
    pandas_df.to_csv(cleaned_csv_file_path, index=False)
    

Shape Before Filtering:  (135, 2)
Removed the row with index:  8
Removed the row with index:  10
Removed the row with index:  12
Removed the row with index:  28
Removed the row with index:  29
Removed the row with index:  30
Removed the row with index:  32
Removed the row with index:  46
Removed the row with index:  53
Removed the row with index:  54
Removed the row with index:  72
Removed the row with index:  77
Removed the row with index:  79
Removed the row with index:  92
Removed the row with index:  94
Removed the row with index:  97
Removed the row with index:  99
Removed the row with index:  105
Removed the row with index:  115
Removed the row with index:  123
Removed the row with index:  124
Removed the row with index:  125
Removed the row with index:  127
Shape After Filtering:  (112, 2)
Shape Before Filtering:  (58, 2)
Removed the row with index:  3
Removed the row with index:  12
Removed the row with index:  29
Removed the row with index:  33
Removed the row with index:  45


In [114]:
# Load the claims from the CSV file
# claims_df = pd.read_csv("./PreprocessedDataset/" + party + "_claims.csv")
# print(claims_df.head())

# ids_to_claims_to_questions = {}

# for index, row in tqdm.tqdm(claims_df.iterrows(), total=claims_df.shape[0]):
#     claims_to_questions = {}
#     # Extract the claims from the DataFrame
#     claims = json.loads(row['claims'])
#     id = row['ad_archive_id']
#     # Extract claims from the text
#     for claim in tqdm.tqdm(claims['Claims']):
#         questions = claim_to_question(claim)
#         # Store the claims in the dictionary
#         claims_to_questions[claim] = questions
#     # Store the claims in the dictionary
#     ids_to_claims_to_questions[id] = claims_to_questions
# # Check if the folder and file exist
# if not os.path.exists("./PreprocessedDataset/"):
#     os.makedirs("./PreprocessedDataset/")
# # Save the claims to a CSV file
# with open("./PreprocessedDataset/" + party + "_claims_to_questions.csv", "w", newline='', encoding='utf-8') as csvfile:
#     fieldnames = ['ad_archive_id', 'claims_to_questions']
#     writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
#     writer.writeheader()
#     for id, claims in ids_to_claims_to_questions.items():
#         writer.writerow({'ad_archive_id': id, 'claims_to_questions': json.dumps(claims)})

saved_questions_path = "./dataset/questions-extraction/"
for party in tqdm.tqdm(parties):
    # Read the CSV file into a DataFrame
    csv_file_path = os.path.join(save_folder_path, party + "_claims_cleaned.csv")
    pandas_df = pd.read_csv(csv_file_path)
    
    # Print the DataFrame shape
    print(f"Shape of {party} DataFrame:", pandas_df.shape)
    
    # Check if the DataFrame is empty
    if pandas_df.empty:
        print(f"The DataFrame for {party} is empty.")
        continue
    
    id_claims_to_questions = {}
    
    # Iterate through the rows of the DataFrame
    iteration = 1
    for index, row in tqdm.tqdm(pandas_df.iterrows(), total=pandas_df.shape[0]):
        print("Iteration: ", iteration)
        iteration += 1
        claims_to_questions = {}
        # Extract the claims from the DataFrame
        claims = row['claims']
        try:
            claims = json.loads(claims)
            c = claims["Claims"]
            id = row['ad_archive_id']
            # Extract claims from the text
            print(claims)
            for claim in claims['Claims']:
                questions = claim_to_question(claim)
                # Store the claims in the dictionary
                claims_to_questions[claim] = questions
            # Store the claims in the dictionary
            id_claims_to_questions[id] = claims_to_questions
            print(claims_to_questions)
        except:
            # Remove the row from the DataFrame
            pandas_df = pandas_df.drop(index)
            print("Removed the row with index: ", index)
            continue
        
        

    
    # Save the claims to a CSV file
    with open(os.path.join(saved_questions_path, party + "_claims_to_questions.csv"), "w", newline='', encoding='utf-8') as csvfile:
        fieldnames = ['ad_archive_id', 'claims_to_questions']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for id, claims in id_claims_to_questions.items():
            writer.writerow({'ad_archive_id': id, 'claims_to_questions': json.dumps(claims)})
        

  0%|          | 0/5 [00:00<?, ?it/s]

Shape of DUI DataFrame: (112, 2)


Iteration:  1
{'Claims': ['НемаОткажуваќьOdИднината', 'The SDSM is...', 'Macedonia']}


{'НемаОткажуваќьOdИднината': {'Questions': ['What are the key attributes that the next President of North Macedonia should possess?', 'How do the presidential candidates differ in their promises and slogans?', 'What specific policies or initiatives do the seven aspirants for the presidency promise to implement if elected?']}, 'The SDSM is...': {'Questions': ['What is the SDSM?', 'What are the attributes that the next President of North Macedonia should possess?', 'What promises and slogans do the presidential candidates offer to attract voters?']}, 'Macedonia': {'Questions': ['What is the current situation regarding the presidential election in North Macedonia?', 'Are there any specific requirements for the next president of North Macedonia, as mentioned in the claim?', "Who are the seven candidates competing for the presidency in North Macedonia's election?"]}}
Iteration:  2
{'Claims': ['SDSM is the only party ready to fight for citizens and the future of the state.', 'The party has e

{'SDSM is the only party ready to fight for citizens and the future of the state.': {'Questions': ["Is the claim 'SDSM is the only party ready to fight for citizens and the future of the state' supported by factual evidence?", 'What specific attributes should the next President of North Macedonia possess, according to the author Enis Shaqiri?', 'What are the concrete offers made by the presidential candidates Biljana Vankovski, Maksim Dimitrievski, and Stefçe Jakimovski?']}, 'The party has endured time and weather, good and bad, and has always been on guard for the homeland.': {'Questions': ['Is the party mentioned in the claim a political party?', 'What are the specific promises and slogans offered by the presidential candidates?', "What is the relevance of the phrase 'homeland' to the context of the presidential election?"]}, "It's not an option for us to give up; SDSM is a party made of solid material.": {'Questions': ['What is the context of the claim that SDSM is a party made of s

...

{'Hungary is the richest country in Europe': {'Questions': ['What is the source of the claim that Hungary is the richest country in Europe?', 'Who are the candidates running for President of North Macedonia and what promises do they offer?', 'What is the significance of the question about the attributes required by the next President of North Macedonia?']}, 'The Social Democratic Party of Macedonia (LSDM) and the Democratic Union for Integration (BDI) are going into opposition': {'Questions': ['What political parties are going into opposition in Macedonia?', 'What are the key differences between the presidential candidates in the Macedonian presidential election?', 'What is the current situation with the presidential election in North Macedonia and what are the main challenges for the future president?']}}
Iteration:  17
{'Claims': ['A new church is being built in Prevalec', 'New sports facilities and urban equipment are being installed in Prevalec', 'The area will be urbanized and dev

{'A new church is being built in Prevalec': {'Questions': ['Is it true that a new church is being built in Prevalec?', 'What specific attributes should the next President of North Macedonia possess, according to the claim?', 'Do the presidential candidates really offer a wide range of promises and slogans to attract voters?']}, 'New sports facilities and urban equipment are being installed in Prevalec': {'Questions': ['Is the claim that new sports facilities and urban equipment are being installed accurate?', 'What is the specific role or attribute required of the future President of North Macedonia, as mentioned in the claim?', 'Do the presidential candidates actually offer a range of promises and slogans to attract voters?']}, 'The area will be urbanized and developed, providing citizens with leisure activities and sports options': {'Questions': ["What is the claim referring to with 'The area will be urbanized and developed'?", 'What kind of leisure activities and sports options are 

{'The city of Veles would not be what it is without these market vendors.': {'Questions': ['What is the relationship between the market vendors and the city of Veles?', 'How do presidential candidates in North Macedonia typically campaign for office?', 'What are the main issues or promises being made by the seven presidential aspirants competing in the election?']}, 'Citizens are disillusioned with this government and the economic catastrophe they have caused.': {'Questions': ["Is the statement 'Citizens are disillusioned with this government and the economic catastrophe they have caused' supported by evidence?", 'What specific attributes should a future President of North Macedonia possess, according to the current situation?', 'Do the presidential candidates in the 2023 election offer a range of promises and slogans that aim to attract voters?']}, 'We will need a strong battle to stabilize the economy.': {'Questions': ['What is the claim stating as a necessary condition for stabilizi

100%|██████████| 5/5 [16:33<00:00, 198.60s/it]

{'The candidate is a member of VMRO-DPMNE': {'Questions': [['The VMRO-DPMNE party has a member running for president in the election.', 'Is the candidate indeed a member of VMRO-DPMNE?', 0], ['What is the main dilemma in the presidential election in North Macedonia?', 'Can you identify what the major issue is in the electoral arena?', 1], ['Are there more than three candidates running for the presidential position in the election?', 'Count the number of aspirants mentioned and answer this question.', 2]]}, 'The candidate represents the interests of the Macedonian people': {'Questions': [['Is the claim stating that a candidate represents the interests of the Macedonian people?', 'True'], ['Does the article mention a specific date for the start of the election campaign?', 'False'], ['Are there more than 3 candidates running in the presidential election?', 'Unknown']]}, 'The candidate will work to preserve and strengthen national identity': {'Questions': ['How does the candidate plan to p

In [ ]:
read_path = "./dataset/questions-extraction/"

# Read all the claims to questions and remove the rows which are empty
for party in parties:
    # Read the CSV file into a DataFrame
    csv_file_path = os.path.join(read_path, party + "_claims_to_questions.csv")
    pandas_df = pd.read_csv(csv_file_path)
    print("Before removing the columns for the Party " + party + " the shape is: ", pandas_df.shape)
    
    question_proofs = {}
    for index, row in pandas_df.iterrows():
        try:
            q = json.loads(row["claims_to_questions"])
            v = q.values()
            for i in v:
                for j in i.values():
                    for k in j:
                        # print(k)
                        continue
        except:
            # Remove the row from the DataFrame
            pandas_df = pandas_df.drop(index)
            # print("Removed the row with index: ", index)
            continue
                

    print("After removing the columns for the Party " + party + " the shape is: ", pandas_df.shape)
    print("\n")
    
    # Save the cleaned DataFrame to a new CSV file
    cleaned_csv_file_path = os.path.join(read_path, party + "_claims_to_questions_cleaned.csv")
    pandas_df.to_csv(cleaned_csv_file_path, index=False)
    

Before removing the columns for the Party DUI the shape is:  (112, 2)
After removing the columns for the Party DUI the shape is:  (102, 2)


Before removing the columns for the Party SDSM the shape is:  (49, 2)
After removing the columns for the Party SDSM the shape is:  (45, 2)


Before removing the columns for the Party Left the shape is:  (70, 2)
After removing the columns for the Party Left the shape is:  (60, 2)


Before removing the columns for the Party VMRO-DPMNE the shape is:  (19, 2)
After removing the columns for the Party VMRO-DPMNE the shape is:  (16, 2)




## Google Search (Evidence) Retrieval

In [27]:
def google_search(query, api_key, cse_id, num=3):
    try:
        query = quote_plus(query)
        url = f"https://www.googleapis.com/customsearch/v1?q={query}&key={api_key}&cx={cse_id}&num={num}"
        response = requests.get(url).json()
        results = []
        for item in response.get('items', []):
            # print(item)
            results.append(item.get('snippet'))
            if item.get('snippet') is None:
                print("\n")
                print(response)
            result = item.get('snippet')
        print(results)
    except:
        print("Error occurred while making the request.")
        results = []
    return results

In [26]:
# Evidence = {}
# Claims_and_Evidence_dict = {}
# for claim, questions in claim_to_question_dic.items():
#     print(claim)
#     print(questions)
#     for question in questions:
#         search_results = google_search(question, google_api, cx)
#         Evidence[question] = search_results
#     Claims_and_Evidence_dict[claim] = Evidence
#     Evidence = {}
    
    
    
#     # for question in questions:
#     #     search_results = google_search(question, google_api, cx)
#     #     Evidence[question] = search_results
#     # Claims_and_Evidence_dict[claim] = Evidence
#     # Evidence = {}
#     # print(Claims_and_Evidence_dict)


read_questions_path = "./dataset/questions-extraction/"
save_question_path = "./dataset/proof-extraction/"
# Extract Proofs from Google
for party in parties:
    # Read the CSV file into a DataFrame
    csv_file_path = os.path.join(read_questions_path, party + "_claims_to_questions_cleaned.csv")
    pandas_df = pd.read_csv(csv_file_path)
    
    
    
    for index, row in tqdm.tqdm(pandas_df.iterrows(), total=pandas_df.shape[0]):
        # try:
        id = row['ad_archive_id']
        rowJson = json.loads(row["claims_to_questions"])
        for key, value in rowJson.items():
            questions = value.get("Questions", [])
            enriched_questions = []
            
            for question in questions:
                search_results = google_search(question, google_api, cx)
                enriched_questions.append({
                    'question': question,
                    'search_results': search_results
                })
                
            rowJson[key]['Questions'] = enriched_questions
            # Update the DataFrame with the enriched questions
            pandas_df.at[index, 'claims_to_questions'] = json.dumps(rowJson)
            print(pandas_df.at[index, 'claims_to_questions'])
            break

            
        # except:
        #     # Remove the row from the DataFrame
        #     pandas_df = pandas_df.drop(index)
        #     print("Removed the row with index: ", index)
        #     # print("Removed the row with index: ", index)
    
        
    # Save the cleaned DataFrame to a new CSV file
    cleaned_csv_file_path = os.path.join(save_question_path, party + "_claims_to_questions_to_proofs.csv")
    pandas_df.to_csv(cleaned_csv_file_path, index=False)
    print(pandas_df.head())
    

  0%|          | 0/102 [00:00<?, ?it/s]

[]
['In those days, the candidate receiving the second-highest vote became the vice president. In a scheme to deny Adams the presidency, Alexander Hamilton\xa0...', 'Jan 11, 1989 ... Well, back in 1980, when I was running for President, it was all so different. Some pundits said our programs would result in catastrophe. Our\xa0...', 'The Campaign and Election of 1912. Although Woodrow Wilson was convinced that God had destined him to be president, it took all his political skill and a good\xa0...']


  1%|          | 1/102 [00:00<00:49,  2.03it/s]

['Nov 6, 2024 ... Donald Trump is set to return to the White House, having promised action on issues including immigration, the economy and the war in Ukraine.', "We do not account for possible changes in GDP resulting from the candidates ... In some cases, we looked to specific policies in past President's budgets and\xa0...", '" Republicans touted Roosevelt\'s record in foreign policy and promised more of the same. Neither Roosevelt nor Parker actively campaigned for the presidency\xa0...']
{"\u041d\u0435\u043c\u0430\u041e\u0442\u043a\u0430\u0436\u0443\u0432\u0430\u045c\u044cOd\u0418\u0434\u043d\u0438\u043d\u0430\u0442\u0430": {"Questions": [{"question": "What are the key attributes that the next President of North Macedonia should possess?", "search_results": []}, {"question": "How do the presidential candidates differ in their promises and slogans?", "search_results": ["In those days, the candidate receiving the second-highest vote became the vice president. In a scheme to deny Ada

  2%|▏         | 2/102 [00:00<00:49,  2.04it/s]

...

['We offer this analysis with humility, knowing that invariably the future will unfold in ways that we have not foreseen. Although Global Trends is necessarily\xa0...', 'With that in mind, the. Illinois Department of Natural Resources continues to work on improving hunting opportunities, recruit new hunters and trappers, and\xa0...', "Given the Macedonia case, villages should consult their ... If the village does not have the authority to operate a mayor's court pursuant to Chapter 1905."]
{"The price we pay for silence and apathy is too high!": {"Questions": [{"question": "How does the text explicitly state that the price paid for silence and apathy is too high?", "search_results": ["I need not pause to say how very delighted I am to be here tonight, and how very delighted I am to see you expressing your concern about the issues that will be\u00a0...", "Nov 20, 2016 ... Many local crises which, in an isolated state, would ... In the Soviet Union, samizdat has a longer tradition and cl

 37%|███▋      | 22/60 [00:19<00:36,  1.05it/s]

['Jun 21, 2017 ... I guess the best way I can describe it is that this was a, my ... Do these services address possible vulnerabilities related to vote.', 'Sep 22, 2024 ... Multilateralism, he said, is the best tool with which to address ... GORDANA SILJANOVSKA-DAVKOVA, President of the Republic of North Macedonia\xa0...', 'Jun 28, 2021 ... ... president thought this would be the right ... The North Macedonians and neighbouring Serbs now rushed to leverage a brief moment of political\xa0...']
{"Na\u00efve is the best way to punish corrupt political elites by voting in elections.": {"Questions": [{"question": "What is the specific claim being made about punishing corrupt political elites through voting?", "search_results": ["Sep 5, 2023 ... ... by those who claim to be most politically knowledgeable. This may ... be an elite problem that had little effect on regular voters. If\u00a0...", "Angry supporters of then-president Donald Trump, believing false accusations that Dominion had swit

 38%|███▊      | 23/60 [00:19<00:34,  1.09it/s]

['May 15, 2024 ... ... election campaign (for example the party promised ... Macedonia held a parliamentary election and the runoff to the presidential election.', 'May 7, 2024 ... Their 2020 election billboards read “Purification of the judiciary” and the main ... proposed by parties and the President must meet. They\xa0...', 'I studied the attitudes of the political elite regarding the common state on the basis of speeches held by leading politicians during election campaigns, at\xa0...']
[]
{"Macedonia's merits are being returned": {"Questions": [{"question": "Is the Macedonian presidential election a current event?", "search_results": ["Aug 25, 2020 ... Many of the most viral posts originated from Veles, a small town in central Macedonia. During fieldwork in Veles, where we interviewed several residents and\u00a0...", "Apr 24, 2024 ... SKOPJE, April 24 (Reuters) - North Macedonian presidential candidate Gordana Siljanovska-Davkova held a big lead over incumbent Stevo\u00a0...", "Ap

In [ ]:
# Print Claims_and_Evidence_dict in a pretty format
print(json.dumps(Claims_and_Evidence_dict, indent=4))

{
    "Himni dhe simbolet shtet\u00ebrore kan\u00eb vendin dhe koh\u00ebn e vet se ku p\u00ebrdoren": {
        "What is the purpose of singing national anthems according to Hristijan Mickoski's proposal?": [
            {
                "title": "North Macedonia's parliament approves a new center-right ...",
                "link": "https://apnews.com/article/north-macedonia-mickoski-government-8021f259cd9b8fd3f128f35f533ab481",
                "snippet": "Jun 23, 2024 ... ... proposed by a center-right party that won May's national elections. ... Hristijan Mickoski, 46, faces significant challenges in his four\u00a0..."
            },
            {
                "title": "MONOETHNIC ANTHEM: Education under the claws of nationalism ...",
                "link": "https://koha.mk/en/himni-monoetnik-arsimi-nen-kthetrat-e-nacionalizmit/",
                "snippet": "Dec 16, 2024 ... The law on secondary education that foresees the singing of the national anthem in schools is the result

## Identify Stance of Evidence

In [96]:
def detect_stance(claim, evidence):
    prompt = f"""Analyze whether this evidence supports or refutes the claim.
    Claim: "{claim}"
    Evidence: "{evidence}"
    Respond only with on of the following options: Completely Supports, Partially Support, Refute, Irrelevant, or No Evidence.
    """
    extract_stance_payload = {
    "model": "llama3",
    "prompt": prompt,
    }
    stance_response = requests.post(llama3_url, json=extract_stance_payload)
    if stance_response.status_code == 200:
        # Process the streamed response
        lines = stance_response.text.splitlines()
        output = ""

        for line in lines:
            try:
                # Parse each line as JSON
                line_data = json.loads(line)
                # Concatenate the "response" field
                output += line_data.get("response", "")
            except json.JSONDecodeError:
                # Skip lines that are not valid JSON
                continue

        # Parse the final concatenated output as JSON
        # Remove everything after the newline character
        output = output.split('\n')[0]
        return output
    else:
        print(f"Error: {stance_response.status_code}")
        print(stance_response.text)
    return None

In [105]:
stance_results = {}


for claim, evidence in tqdm.tqdm(Claims_and_Evidence_dict.items()):
    for question, evidences in tqdm.tqdm(evidence.items()):
        for evidence in tqdm.tqdm(evidences):
            stance = detect_stance(claim, evidence)
            if claim not in stance_results:
                stance_results[claim] = {}
            if question not in stance_results[claim]:
                stance_results[claim][question] = []
            stance_results[claim][question].append({
                'evidence': evidence,
                'stance': stance
            })

100%|██████████| 7/7 [07:07<00:00, 61.02s/it]


In [106]:
# Print stance_results in a pretty format
print(json.dumps(stance_results, indent=4))

{
    "Himni dhe simbolet shtet\u00ebrore kan\u00eb vendin dhe koh\u00ebn e vet se ku p\u00ebrdoren": {
        "What is the purpose of singing national anthems according to Hristijan Mickoski's proposal?": [
            {
                "evidence": {
                    "title": "North Macedonia's parliament approves a new center-right ...",
                    "link": "https://apnews.com/article/north-macedonia-mickoski-government-8021f259cd9b8fd3f128f35f533ab481",
                    "snippet": "Jun 23, 2024 ... ... proposed by a center-right party that won May's national elections. ... Hristijan Mickoski, 46, faces significant challenges in his four\u00a0..."
                },
                "stance": "Partially Supports"
            },
            {
                "evidence": {
                    "title": "MONOETHNIC ANTHEM: Education under the claws of nationalism ...",
                    "link": "https://koha.mk/en/himni-monoetnik-arsimi-nen-kthetrat-e-nacionalizmit/",
   

## Determine Correction

In [ ]:
def determine_verdict(stance_results):
    verdicts = []
    for claim, questions in stance_results.items():
        for question, evidences in questions.items():
            # Make sure that all the stance results are in the same format
            for evidence in evidences:
                for key in evidence:
                    if key == 'stance':
                        stance = evidence['stance']
                        
                        if stance != 'Completely Supports' and stance != 'Partially Support' and stance != 'Refute':
                            evidence['stance'] = 'Irrelevant'
                            # Search if the sentence contains any of the words
                            if 'Completely Supports' in stance:
                                evidence['stance'] = 'Completely Supports'
                            elif 'Partially Support' in stance:
                                evidence['stance'] = 'Partially Support'
                            elif 'Refute' in stance:
                                evidence['stance'] = 'Refute'
                            elif 'Irrelevant' in stance:
                                evidence['stance'] = 'Irrelevant'
                            elif 'No Evidence' in stance:
                                evidence['stance'] = 'No Evidence'
                        print(stance)
                    
            
            supports = sum(1 for evidence in evidences if evidence['stance'] == 'Completely Supports' or evidence['stance'] == 'Partially Support')
            refutes = sum(1 for evidence in evidences if evidence['stance'] == 'Refute')
            if supports > refutes:
                verdicts.append((claim, question, "Supports"))
            elif refutes > supports:
                verdicts.append((claim, question, "Refutes"))
            else:
                verdicts.append((claim, question, "Neutral"))
                
            
            
    return verdicts

In [124]:
determine_verdict(stance_results)

Partially Support
Partially Support
Refute
Partially Support
Partially Support
Partially Support
Partially Support
No Evidence
Irrelevant
Refute
Refute
Refute
Refute
Refute
Refute
Refute
Refute
Partially Support
Partially Support
Partially Support
No Evidence
Partially Support
Partially Support
Partially Support
Partially Support
Irrelevant
Refute
No Evidence
Partially Support
Partially Support
Irrelevant
Irrelevant
Irrelevant
Partially Support
Partially Support
Partially Support
Irrelevant
Partially Support
Partially Support
Partially Support
Partially Support
Partially Support
No Evidence
Partially Support
Partially Support
No Evidence
Partially Support
No Evidence
No Evidence
Partially Support
Partially Support
No Evidence
Partially Support
Partially Support
No Evidence
Irrelevant
Irrelevant


[('Himni dhe simbolet shtetërore kanë vendin dhe kohën e vet se ku përdoren',
  "What is the purpose of singing national anthems according to Hristijan Mickoski's proposal?",
  'Supports'),
 ('Himni dhe simbolet shtetërore kanë vendin dhe kohën e vet se ku përdoren',
  'Is it true that VMRO-DPMNE promotes nationalism and anti-European agendas, as claimed by the author?',
  'Supports'),
 ('Himni dhe simbolet shtetërore kanë vendin dhe kohën e vet se ku përdoren',
  'Does the author suggest that VMRO-DPMNE isolates the country and hinders its future prospects in Europe?',
  'Supports'),
 ('The proposal of the leader of VMRO-DPMNE Hristijan Mickoski to make children sing the national anthem every month reflects the nationalist mindset of VMRO-DPMNE',
  'Is the proposal to make children sing the national anthem every month a common practice in any country?',
  'Refutes'),
 ('The proposal of the leader of VMRO-DPMNE Hristijan Mickoski to make children sing the national anthem every month re

In [125]:
def claims_verdict(claims, stance_results):
    verdicts = []
    for claim in claims:
        supports = sum(1 for question, evidences in stance_results[claim].items() if any(evidence['stance'] == 'Completely Supports' or evidence['stance'] == 'Partially Support' for evidence in evidences))
        refutes = sum(1 for question, evidences in stance_results[claim].items() if any(evidence['stance'] == 'Refute' for evidence in evidences))
        if supports > refutes:
            verdicts.append((claim, "Supports"))
        elif refutes > supports:
            verdicts.append((claim, "Refutes"))
        else:
            verdicts.append((claim, "Neutral"))
    return verdicts
claims_verdict(claims, stance_results)

[('Himni dhe simbolet shtetërore kanë vendin dhe kohën e vet se ku përdoren',
  'Supports'),
 ('The proposal of the leader of VMRO-DPMNE Hristijan Mickoski to make children sing the national anthem every month reflects the nationalist mindset of VMRO-DPMNE',
  'Refutes'),
 ('VMRO-DPMNE spreads unjust nationalism while making excuses for anti-European agendas',
  'Supports'),
 ('VMRO-DPMNE fosters hatred and division', 'Supports'),
 ('VMRO-DPMNE blocks the European future of young generations', 'Supports'),
 ('VMRO-DPMNE isolates the country', 'Supports'),
 ('VMRO-DPMNE exercises power against Albanian nation in the municipalities that lead its quotas',
  'Supports')]